<a href="https://colab.research.google.com/github/RachelRFode/grownome-clean/blob/master/2018_12_07_general.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Summary
Use shell and python
Get list of all of the tiles
Function takes one of those file names into device id, timestamp, hash (python)

Imports

In [0]:
import os
import pandas as pd
# using timestamp parsing function
from datetime import datetime
# sorting function
from operator import itemgetter, attrgetter
# time
from datetime import datetime, timedelta
# csv exporter
import csv

Authenticate google to obtain data

In [0]:
# Login to grab data from google cloud
project_id = 'grownome'
from google.colab import auth
auth.authenticate_user()

OPTIONAL

In [0]:
!gsutil -m rsync -r gs://grownome.appspot.com/images/ gs://grownome-vcm/images/

Building synchronization state...
At source listing 10000...
At destination listing 10000...
At destination listing 20000...
Starting synchronization...
Copying gs://grownome.appspot.com/images/2538009072846140/1544230328755-0644ed21611a1d9a3ea068088a492b86.jpg [Content-Type=image/jpeg]...
Copying gs://grownome.appspot.com/images/2538009072846140/1544370062635-062f6c1f08fd38641b5f1cb2a5492d30.jpg [Content-Type=image/jpeg]...
Copying gs://grownome.appspot.com/images/2538009072846140/1544383911935-0502087a3c4d9a5c2ec430bf0f94a129.jpg [Content-Type=image/jpeg]...
Copying gs://grownome.appspot.com/images/2538009072846140/1544358865658-072bf0ee7aff82de7fecf740687e47ce.jpg [Content-Type=image/jpeg]...
Copying gs://grownome.appspot.com/images/2538009072846140/1544235874323-075902e70344c0d0ebf03a80c0414712.jpg [Content-Type=image/jpeg]...
Copying gs://grownome.appspot.com/images/2538009072846140/1544353116120-02acc88156d878447dd35d03cf2aa9c8.jpg [Content-Type=image/jpeg]...
Copying gs://growno

Pull images from folder in Google Cloud

In [0]:
# Get data from google cloud  and assign it to list
!gsutil ls gs://grownome-vcm/images/2538009072846140/ > imagelist.txt
  
#check
#!cat imagelist.txt

Pull data

In [0]:
def parse_img_list(filepath):
  # create list for raw images
  raw_img_paths = []

  # get handle to txt file, read lines out of txt file, 
  # split by '\n', close handle
  with open(filepath) as f:
    raw_img_paths = f.read().split('\n')

  # create list for img paths
  parsed_img_paths = []

  # parsing img path filenames
  for img_path in raw_img_paths:
    if img_path != '':
      # split the url
      split_img_path = img_path.split('/')
      # get last device id and img filename from url
      device_id = split_img_path[4]
      file_name = split_img_path[5]
      # split img file name by '-'
      file_name_parts = file_name.split('-')
      
      if(len(file_name_parts[0]) == 13):
        # convert timestamp to int
        # divide timestamp by 1000 to convert from miliseconds to seconds
        timestamp = int(file_name_parts[0])/1000
        # parse timestamp into time object
        parsed_timestamp = datetime.utcfromtimestamp(timestamp)
        # putting device_id, timestamp, and img_path into dictionary
        parsed_part = {'device_id': device_id, 
                       'timestamp': parsed_timestamp,
                       'img_path': img_path}
        # append parsed part to list
        parsed_img_paths.append(parsed_part)

  # check: print list
  return parsed_img_paths

unsorted_img_list = parse_img_list('imagelist.txt')

Sort list by timestamp

In [0]:
dark_img_list = sorted(unsorted_img_list, key=itemgetter('timestamp'))

In [0]:
dark_img_list

with open('dark_img_list.csv', 'w') as csvfile:
    fieldnames = ['img_path']
    
    writer = csv.DictWriter(csvfile, extrasaction='ignore', fieldnames=fieldnames)

    for item in dark_img_list:
      
      writer.writerow(item)

In [0]:
!gsutil cp ./dark_img_list.csv gs://grownome-vcm/files.csv

Copying file://./dark_img_list.csv [Content-Type=text/csv]...
-
Operation completed over 1 objects/418.0 KiB.                                    


Upload File to Google - AutoML Vision


*   make data set that assumes that plants with low humidity are wilted
*   label them all as low, med, high (add a label into each item of the list)
*  print into csv file
*  upload to google cloud (gsutil)

also try the humidity 3 hrs ago

you havd 7:00 timestamps - if humid is high and it is 7:00 am assume it is watered
